In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import cv2
import tensorflow as tf
from keras.preprocessing import image_dataset_from_directory
from keras.preprocessing.image import img_to_array
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, MaxPool2D
import glob
import os
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# Mengimpor modul yang diperlukan
import os
import warnings;
warnings.filterwarnings('ignore');

### Load Dataset

In [ ]:
data_dir= r'G:\My Drive\Semester 2\Advanced Deep Learning\UTS\archive'

### Data Prepration And Preprocessing

In [ ]:
images_path=glob.glob(data_dir+'/**/*.jpg',recursive=True, root_dir=data_dir)

In [ ]:
images_path[:5]

In [ ]:
labels= []
for img in images_path :
    lab= os.path.dirname(img)
    labels.append(lab)

In [ ]:
image_labels=[]
for label in labels:
    lab=label.split('/')[-1]
    image_labels.append(lab)

In [ ]:
image_labels[:10]

In [ ]:
preprocessed_images = []


for file in images_path:
    img = cv2.imread(file)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (128,128))
    img = img / 255
    
    preprocessed_images.append(img)

In [ ]:
X= np.array(preprocessed_images)

In [ ]:
X.shape

In [ ]:
# visualize sample images
attribute = image_dataset_from_directory(data_dir, image_size=(100,100))
plt.figure(figsize=(20,25))
for image, labels in attribute.take(1):
  for i in range (25):
    plt.subplot(5,5, i + 1)
    plt.imshow(np.array(image[i]).astype("uint8"))
    plt.title(attribute.class_names[labels[i]])
    plt.axis("off")

In [ ]:
y = np.array(image_labels)

In [ ]:
encoder = LabelEncoder()
y = encoder.fit_transform(y)

### Splittin Data

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=42,shuffle=True)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

## Modelling

### Model VGG-16 tanpa transfer learning

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

def VGG16():
    model = Sequential()

    # Block 1
    model.add(Conv2D(64, (3, 3), activation='relu', padding='same', input_shape=(128, 128, 3)))
    model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))

    # Block 2
    model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
    model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))

    # Block 3
    model.add(Conv2D(256, (3, 3), activation='relu', padding='same'))
    model.add(Conv2D(256, (3, 3), activation='relu', padding='same'))
    model.add(Conv2D(256, (3, 3), activation='relu', padding='same'))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))

    # Block 4
    model.add(Conv2D(512, (3, 3), activation='relu', padding='same'))
    model.add(Conv2D(512, (3, 3), activation='relu', padding='same'))
    model.add(Conv2D(512, (3, 3), activation='relu', padding='same'))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))

    # Block 5
    model.add(Conv2D(512, (3, 3), activation='relu', padding='same'))
    model.add(Conv2D(512, (3, 3), activation='relu', padding='same'))
    model.add(Conv2D(512, (3, 3), activation='relu', padding='same'))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))

    # Flatten
    model.add(Flatten())

    # Dense layers
    model.add(Dense(4096, activation='relu'))
    model.add(Dense(4096, activation='relu'))
    model.add(Dense(1000, activation='softmax'))  # Assuming 1000 classes for ImageNet

    return model

# Create the model
transfer_learning_model = VGG16()

# Display model summary
transfer_learning_model.summary()

In [ ]:
history = transfer_learning_model.fit(X_train, y_train, epochs=50, batch_size=100, validation_data=(X_val, y_val))

In [ ]:
# Ambil loss dari history
train_loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(train_loss) + 1)

In [ ]:
# Plot kurva loss
plt.plot(epochs, train_loss, 'g', label='Training Loss')
plt.plot(epochs, val_loss, 'b', label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
# Mengevaluasi kinerja model menggunakan data validasi
val_loss, val_acc = transfer_learning_model.evaluate(X_val, y_val)
print("Model Validation Accuracy:", val_acc)

### Evaluating the Model

In [ ]:
# Evaluate the model on the test set
test_loss, test_accuracy = transfer_learning_model.evaluate(X_test,y_test)
print(f'Test loss: {test_loss}')
print(f'Test accuracy: {test_accuracy}')

### Transfer Learning without Fine Tuning

In [ ]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense

def create_transfer_learning_model(input_shape, num_classes):
    base_model = VGG16(weights='imagenet', include_top=False, input_shape=input_shape)

    # Freeze the base model
    base_model.trainable = False

    # Add custom layers on top
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(1024, activation='relu')(x)
    predictions = Dense(num_classes, activation='softmax')(x)

    # This is the model we will train
    model = Model(inputs=base_model.input, outputs=predictions)
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

# 6 classes for the fruits
transfer_learning_model = create_transfer_learning_model((128, 128, 3), 6)
transfer_learning_model.summary()

In [ ]:
history = transfer_learning_model.fit(X_train, y_train, epochs=50, batch_size=100, validation_data=(X_val, y_val))

In [ ]:
# Ambil loss dari history
train_loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(train_loss) + 1)

In [ ]:
# Plot kurva loss
plt.plot(epochs, train_loss, 'g', label='Training Loss')
plt.plot(epochs, val_loss, 'b', label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
# Mengevaluasi kinerja model menggunakan data validasi
val_loss, val_acc = transfer_learning_model.evaluate(X_val, y_val)
print("Model Validation Accuracy:", val_acc)

### Evaluating the Model

In [ ]:
# Evaluate the model on the test set
test_loss, test_accuracy = transfer_learning_model.evaluate(X_test,y_test)
print(f'Test loss: {test_loss}')
print(f'Test accuracy: {test_accuracy}')

### Transfer Learning with Fine Tuning

In [ ]:
# Define the base model
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(128, 128, 3))

# Unlock some layers for fine-tuning
base_model.trainable = True

# Fine-tune from this layer onwards
fine_tune_at = 100

# Freeze all layers before the fine-tune layer
for layer in base_model.layers[:fine_tune_at]:
    layer.trainable = False

# Add custom layers on top
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(6, activation='softmax')(x)

# This is the model we will train
transfer_learning_model = Model(inputs=base_model.input, outputs=predictions)

# Compile the model
transfer_learning_model.compile(optimizer='adam',
                                loss='sparse_categorical_crossentropy',
                                metrics=['accuracy'])


In [ ]:
history = transfer_learning_model.fit(X_train, y_train, epochs=50, batch_size=100, validation_data=(X_val, y_val))

In [ ]:
# Ambil loss dari history
train_loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(train_loss) + 1)

In [ ]:
# Plot kurva loss
plt.plot(epochs, train_loss, 'g', label='Training Loss')
plt.plot(epochs, val_loss, 'b', label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
# Mengevaluasi kinerja model menggunakan data validasi
val_loss, val_acc = transfer_learning_model.evaluate(X_val, y_val)
print("Model Validation Accuracy:", val_acc)

### Evaluating the Model

In [ ]:
# Evaluate the model on the test set
test_loss, test_accuracy = transfer_learning_model.evaluate(X_test,y_test)
print(f'Test loss: {test_loss}')
print(f'Test accuracy: {test_accuracy}')